In [1]:
import numpy as np
import sympy as sp

In [2]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [3]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

Matrix([
[x1, x2],
[x3, x4]])

In [4]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [5]:
kronecker_binary([[1,0],[0,1]])

array([0, 1, 0, 0])

In [6]:
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(4), 2)))
variaveis

Matrix([
[x1*x3],
[x1*x4],
[x2*x3],
[x2*x4]])

In [7]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [8]:
numeros_decimais = [0,2,2,1]
vetores_binarios = []
tamanho_bits = int(len(numeros_decimais)/2)

for numero in numeros_decimais:
    try:
        vetor = decimal_para_binario_vetor(numero, tamanho_bits)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 1, 0],
[0, 0, 0, 0]])

In [9]:
vetores_binarios

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0]])

In [10]:
# Lista para armazenar os índices de cada linha
resultado = []

# Percorre cada linha da matriz
for linha in vetores_binarios:
    indices = [i for i, val in enumerate(linha) if val == 1]
    resultado.append(indices if indices else [None])
    
resultado

[[0], [3], [1, 2], [None]]

In [31]:
regra = sp.Matrix(np.dot(vetores_binarios, kronecker_binary(np.array_split(criar_vetor_de_simbolos(4), 2)))) # expressões simbólicas
regra

Matrix([
[        x1*x3],
[        x2*x4],
[x1*x4 + x2*x3],
[            0]])

In [12]:
saidas = regra[0::2]
saidas

[x1*x3, x1*x4 + x2*x3]

In [14]:
def criar_substituicoes(bits):
    simbolos = criar_vetor_de_simbolos(bits * 2)
    substituicoes = {simbolos[i+1]: -simbolos[i] for i in range(0, len(simbolos), 2)}
    return substituicoes

# Aplica substituições
substituicoes = criar_substituicoes(2)
variaveis_substituidas = variaveis.subs(substituicoes)
variaveis_substituidas

Matrix([
[ x1*x3],
[-x1*x3],
[-x1*x3],
[ x1*x3]])

In [33]:
def criar_substituicoes(bits):
    simbolos = criar_vetor_de_simbolos(bits * 2)
    substituicoes = {simbolos[i+1]: 1-simbolos[i] for i in range(0, len(simbolos), 2)}
    return substituicoes

In [34]:
substituicoes = criar_substituicoes(2)
substituicoes

{x2: 1 - x1, x4: 1 - x3}

In [41]:
f = sp.simplify(regra.subs(substituicoes))
f

Matrix([
[             x1*x3],
[ (x1 - 1)*(x3 - 1)],
[-2*x1*x3 + x1 + x3],
[                 0]])

In [42]:
regra[0::2]

[x1*x3, x1*x4 + x2*x3]

In [43]:
regra[1::2]

[x2*x4, 0]

In [39]:
from itertools import product

# Passo 3: Gerar todas as combinações de 0 e 1 para as 3 variáveis
entradas = list(product([0, 1], repeat=3))
sp.Matrix(entradas)

Matrix([
[0, 0, 0],
[0, 0, 1],
[0, 1, 0],
[0, 1, 1],
[1, 0, 0],
[1, 0, 1],
[1, 1, 0],
[1, 1, 1]])

In [ ]:
x1, x3, x5 = sp.symbols('x1 x3 x5')

print("x1 x3 x5 | f(x1, x3, x5)")
print("-------------------------")
for vals in entradas:
    valor = f[0].subs({x1: vals[0], x3: vals[1], x5: vals[2]})
    valor = int(valor)  # Garante que o valor seja 0 ou 1
    print(f"{vals[0]}  {vals[1]}  {vals[2]}  |    {valor}")

In [ ]:
from sympy import symbols
from sympy import symbols, Matrix, simplify_logic

def criar_mapping(bits):
    """
    Cria um dicionário de mapeamento dinâmico para substituir as variáveis x1, x2, ..., x(2*bits)
    usando novas variáveis v1, v2, ..., v(bits).

    Parameters:
    bits (int): O número de bits, determinando 2*bits variáveis a serem mapeadas.

    Returns:
    dict: Dicionário contendo as substituições de x_i em termos de v_i.
    """
    # Criação das variáveis originais x1, x2, ..., x(2*bits)
    variaveis_originais = symbols(f'x1:{2*bits+1}')
    
    # Criação das novas variáveis v1, v2, ..., v(bits)
    novas_variaveis = symbols(f'v1:{bits+1}')
    
    # Construção do mapeamento dinâmico
    mapping = {}
    for i in range(bits):
        x_impar = variaveis_originais[2 * i]    # x1, x3, x5, ...
        x_par = variaveis_originais[2 * i + 1]  # x2, x4, x6, ...
        v = novas_variaveis[i]  # v1, v2, v3, ...

        # Mapeamento baseado no padrão identificado
        mapping[x_impar] = 1 - v
        mapping[x_par] = v

    return mapping

# Testando a função com 3 bits
bits = 2
mapping = criar_mapping(bits)

# Aplica a substituição na matriz
M_transformed = regra.subs(mapping)

# Extrai as expressões resultantes
expr_list = [simplify_logic(expr, force=True) for expr in M_transformed]

# Função para calcular a tabela-verdade de uma expressão
def get_truth_table(expr, variables=[x, y]):
    tt = {}
    for vals in itertools.product([0, 1], repeat=len(variables)):
        assignment = dict(zip(variables, vals))
        val = int(bool(expr.subs(assignment)))
        tt[vals] = val
    return tt

# Função para comparar duas tabelas-verdade
def tt_equal(tt1, tt2):
    return all(tt1[k] == tt2[k] for k in tt1)



# Procura quais das expressões da matriz correspondem a cada função
sum_candidates = []
carry_candidates = []

for i, expr in enumerate(expr_list):
    tt_expr = get_truth_table(expr)
    print(f"Expressão {i+1}: {expr}, tabela-verdade: {tt_expr}")
    if tt_equal(tt_expr, tt_expected_sum):
        sum_candidates.append(expr)
    if tt_equal(tt_expr, tt_expected_carry):
        carry_candidates.append(expr)

# Se houver mais de um candidato para cada saída, combina-os com OR
if sum_candidates:
    final_sum = sum_candidates[0]
    for expr in sum_candidates[1:]:
        final_sum = simplify_logic(Or(final_sum, expr), force=True)
else:
    final_sum = None

if carry_candidates:
    final_carry = carry_candidates[0]
    for expr in carry_candidates[1:]:
        final_carry = simplify_logic(Or(final_carry, expr), force=True)
else:
    final_carry = None

print("\nExpressão final da soma (Sum):", final_sum)
print("Expressão final do carry:", final_carry)

NameError: name 'x' is not defined

In [ ]:
from sympy import symbols, Matrix, simplify_logic
from sympy.logic.boolalg import Or
import itertools


# Extrai as expressões resultantes
expr_list = [simplify_logic(expr, force=True) for expr in M_transformed]

# Função para calcular a tabela-verdade de uma expressão
def get_truth_table(expr, variables=[x, y]):
    tt = {}
    for vals in itertools.product([0, 1], repeat=len(variables)):
        assignment = dict(zip(variables, vals))
        val = int(bool(expr.subs(assignment)))
        tt[vals] = val
    return tt

# Função para comparar duas tabelas-verdade
def tt_equal(tt1, tt2):
    return all(tt1[k] == tt2[k] for k in tt1)

# Define as funções esperadas:
expected_sum = simplify_logic(x + y - 2*x*y, force=True)  # XOR
expected_carry = simplify_logic(x*y, force=True)  # AND

tt_expected_sum = get_truth_table(expected_sum)
tt_expected_carry = get_truth_table(expected_carry)

# Procura quais das expressões da matriz correspondem a cada função
sum_candidates = []
carry_candidates = []

for i, expr in enumerate(expr_list):
    tt_expr = get_truth_table(expr)
    print(f"Expressão {i+1}: {expr}, tabela-verdade: {tt_expr}")
    if tt_equal(tt_expr, tt_expected_sum):
        sum_candidates.append(expr)
    if tt_equal(tt_expr, tt_expected_carry):
        carry_candidates.append(expr)

# Se houver mais de um candidato para cada saída, combina-os com OR
if sum_candidates:
    final_sum = sum_candidates[0]
    for expr in sum_candidates[1:]:
        final_sum = simplify_logic(Or(final_sum, expr), force=True)
else:
    final_sum = None

if carry_candidates:
    final_carry = carry_candidates[0]
    for expr in carry_candidates[1:]:
        final_carry = simplify_logic(Or(final_carry, expr), force=True)
else:
    final_carry = None

print("\nExpressão final da soma (Sum):", final_sum)
print("Expressão final do carry:", final_carry)

Expressão 1: (1 - x)*(1 - y), tabela-verdade: {(0, 0): 1, (0, 1): 0, (1, 0): 0, (1, 1): 0}
Expressão 2: x*y, tabela-verdade: {(0, 0): 0, (0, 1): 0, (1, 0): 0, (1, 1): 1}
Expressão 3: x*(1 - y) + y*(1 - x), tabela-verdade: {(0, 0): 0, (0, 1): 1, (1, 0): 1, (1, 1): 0}
Expressão 4: 0, tabela-verdade: {(0, 0): 0, (0, 1): 0, (1, 0): 0, (1, 1): 0}

Expressão final da soma (Sum): x*(1 - y) + y*(1 - x)
Expressão final do carry: x*y


In [ ]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)

funcoes

[<function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>]

In [ ]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = lambdastr(variaveis, expr)
    funcoes.append(func)

funcoes

['lambda x1,x2,x3,x4: (x1*x3)',
 'lambda x1,x2,x3,x4: (x2*x4)',
 'lambda x1,x2,x3,x4: (x1*x4 + x2*x3)',
 'lambda x1,x2,x3,x4: (0)']

In [ ]:
valores = np.array([decimal_para_binario_vetor(0,1),decimal_para_binario_vetor(0,1)]).flatten()
print(valores)

# Calcular os resultados
[f(*valores) for f in funcoes]

[1 0 1 0]


[1, 0, 0, 0]

In [ ]:
sp.Matrix(np.dot(vetores_binarios, [0,1,0,0]))

Matrix([
[0],
[0],
[1],
[0]])

In [ ]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=2))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

[1 0 1 0] [1, 0, 0, 0]
[1 0 0 1] [0, 0, 1, 0]
[0 1 1 0] [0, 0, 1, 0]
[0 1 0 1] [0, 1, 0, 0]
